In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [ ]:
cd {PATH_ROOT}

In [ ]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from src.visualization import specto_vis
from termcolor import colored

from src.data import feat_data, get_data
from src.features import specto_feat
from src.models import arch_setup, base_base_model, alex_model, tcn_model
from tqdm import tqdm

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
train_path = 'MAFAT RADAR Challenge - Training Set V1'
train_dict = get_data.load_data(train_path, PATH_DATA)

#split Tracks here to only do augmentation on Train set
# train_dict, val_dict = get_data.split_train_val_as_df(training_dict,ratio= 6)

In [ ]:
synth_dict = get_data.load_data(PATH_DATA + 'MAFAT RADAR Challenge - Auxiliary Synthetic Set V2')
synth_dict['segment_id'] = np.array(synth_dict['segment_id'].tolist()) - 2000000

In [ ]:
train_dict = specto_feat.data_preprocess(train_dict)
synth_dict = specto_feat.data_preprocess(synth_dict)

In [ ]:
def combine_dicts(train_dict, synth_dict):
    td = dict()
    sd = dict()

    seg_id = train_dict['segment_id']
    iq_sb = train_dict['iq_sweep_burst']

    for a,b in zip(seg_id,iq_sb):
        td[a] = b

    seg_id = synth_dict['segment_id']
    iq_sb = synth_dict['iq_sweep_burst']

    for a,b in zip(seg_id,iq_sb):
        sd[a] = b

    comb_dict = dict()

    for key1,val1 in td.items():
        if key1 in sd.keys():
                val2 = sd[key1]
                comb_dict[key1] = dict()
                comb_dict[key1]['train_iq'] = val1
                comb_dict[key1]['synth_iq'] = val2
    
    return comb_dict

In [ ]:
comb_dict = combine_dicts(train_dict, synth_dict)
idx_val = np.random.choice(list(comb_dict.keys()), int(len(comb_dict.keys()) * 0.2), replace=False)
idx_train = list(set(list(comb_dict.keys())).difference(set(idx_val)))

In [ ]:
train_og = np.ndarray((1,126,32)) 
train_synth = np.ndarray((1,126,32)) 
val_og = np.ndarray((1,126,32)) 
val_synth = np.ndarray((1,126,32)) 

for key,val in comb_dict.items():
    if key in idx_train:
        train_og = np.concatenate((train_og, np.expand_dims(val['train_iq'],0)))
        train_synth = np.concatenate((train_synth, np.expand_dims(val['synth_iq'],0)))
    elif key in idx_val:
        val_og = np.concatenate((val_og, np.expand_dims(val['train_iq'],0)))
        val_synth = np.concatenate((val_synth, np.expand_dims(val['synth_iq'],0)))
    else:
        print(key)

train_og = train_og[1:,:,:]
train_synth = train_synth[1:,:,:]
val_og = val_og[1:,:,:]
val_synth = val_synth[1:,:,:]

In [ ]:
print(train_og.shape, val_synth.shape)

In [ ]:
class DS(Dataset):
    def __init__(self,clean_df,noisy_df):
        super().__init__()
        self.clean_df = clean_df
        self.noisy_df = noisy_df

    def __len__(self):
        return self.noisy_df.shape[0]
        
    def __getitem__(self, idx):
        x = self.noisy_df[idx]
        x_hat = self.clean_df[idx]
        return x, x_hat

In [ ]:
train_set = DS(train_og,train_synth)
val_set = DS(val_og,val_synth)
train_loader = DataLoader(dataset= train_set, batch_size = 32, shuffle = True, num_workers = 2)
val_loader = DataLoader(dataset= val_set, batch_size = 32, shuffle = True, num_workers = 2)

In [ ]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)  
        self.conv2 = nn.Conv2d(32, 1, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(1, 32, 2, stride=2, output_padding=(1,0))
        self.t_conv2 = nn.ConvTranspose2d(32, 1, 2, stride=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = self.t_conv2(x)
        return x

def sparse_loss(autoencoder, images):
    loss = 0
    values = images
    for i in range(len(model_children)):
        values = F.relu((model_children[i](values)))
        loss += torch.mean(torch.abs(values))
    return loss

In [ ]:
class ConvAutoencoder2(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(1, 16, 5, padding=2)  
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = nn.Conv2d(32,64,5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(64, 32, 2, stride=2, output_padding=(1,0))
        self.t_conv2 = nn.ConvTranspose2d(32, 16, 2, stride=2, output_padding=(1,0))
        self.t_conv3 = nn.ConvTranspose2d(16,1,2, stride=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.pool(self.conv3(x))
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = self.t_conv3(x)
        return x

def sparse_loss(model):
    model_children = list(model.children())
    loss = 0
    for i in range(len(model_children)):
        values = F.relu((model_children[i](values)))
        loss += torch.mean(torch.abs(values))
    return loss

In [ ]:
model = ConvAutoencoder().to(device)
criterion=nn.MSELoss()
optimizer=optim.SGD(model.parameters(),lr=1e-3)

In [ ]:
from torchsummary import summary
summary(model, (1, 126, 32))

In [ ]:
# for dirty, clean in train_loader:
#     dirty = dirty.to(device).unsqueeze(1).type(torch.FloatTensor)
#     out = model(dirty)

#     print("Model input / output")
#     print(dirty.shape)
#     print(out.shape)
#     break

In [ ]:
epochs=100

l=len(train_loader)
v=len(val_loader)
losslist=list()
valloss = list()

epochloss=0
running_loss=0
running_val_loss = 0
reg_param = 1e-3

add_sparsity = False
add_regularity = True

for epoch in range(epochs):
  
  print("Entering Epoch: ",epoch)
  for dirty,clean in tqdm((train_loader)):
    dirty = dirty.to(device).unsqueeze(1).type(torch.FloatTensor)
    clean = clean.to(device).unsqueeze(1).type(torch.FloatTensor)

    output=model(dirty)
    mse_loss=criterion(output,clean)
    optimizer.zero_grad()

    if add_sparsity:
      l1_loss = sparse_loss(model, output)
      loss = mse_loss + reg_param * l1_loss
    else:
      loss = mse_loss

    if add_regularity:
      l1_regularization, l2_regularization = torch.tensor(0).type(torch.FloatTensor), torch.tensor(0).type(torch.FloatTensor)
      for param in model.parameters():
        l1_regularization += torch.norm(param, 1)**2
        l2_regularization += torch.norm(param, 2)**2

      loss = loss + l1_regularization + l2_regularization


    loss.backward()
    optimizer.step()    
    running_loss+=loss.item()
    epochloss+=loss.item()

  for dirty,clean in tqdm((val_loader)):
    dirty = dirty.to(device).unsqueeze(1).type(torch.FloatTensor)
    clean = clean.to(device).unsqueeze(1).type(torch.FloatTensor)

    output=model(dirty)
    mse_loss = criterion(output, clean)

    if add_sparsity:
        l1_loss = sparse_loss(model, output)
        vloss = mse_loss + reg_param * l1_loss
    else:
        vloss = mse_loss
    
    running_val_loss += vloss.item()

  #-----------------Log-------------------------------
  losslist.append(running_loss / l)
  valloss.append(running_val_loss / v)
  running_loss = 0
  running_val_loss = 0
  print("======> epoch: {}/{}, Loss:{}\tVal Loss:{}".format(epoch,epochs,loss.item(),vloss.item()))

In [ ]:
plt.figure(figsize=(12,8))
plt.title(model._get_name())
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(len(losslist)),losslist,label="Train")
plt.plot(range(len(valloss)),valloss,label="Val")
plt.legend()
plt.tight_layout()
plt.show();

## Visualize how well model does

In [ ]:
idx = np.random.choice(list(comb_dict.keys()),1)[0]
train = comb_dict[idx]['train_iq']
synth = comb_dict[idx]['synth_iq']
cleaned = torch.from_numpy(train).to(device).view(1,1,126,32).type(torch.FloatTensor)
cleaned = model(cleaned).view(126,32).detach().numpy()

In [ ]:
specto_vis.plot_spectrogram(train,None,color_map_path="/home/shaul/workspace/GitHub/sota-mafat-radar/data/cmap.npy", figsize=(12,8));

In [ ]:
specto_vis.plot_spectrogram(synth,None,color_map_path="/home/shaul/workspace/GitHub/sota-mafat-radar/data/cmap.npy", figsize=(12,8));

In [ ]:
specto_vis.plot_spectrogram(cleaned,None,color_map_path="/home/shaul/workspace/GitHub/sota-mafat-radar/data/cmap.npy", figsize=(12,8));